In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jupyter_black

jupyter_black.load()

In [3]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np

from acoustics.constants import primes, speed_of_sound

# Generator

The following generates a standard 2d QRD sequence with a specified width, height, prime, and offset. 

The width, height, and prime chosen should generally result in a symetrical layout.

The offset, k, can be any number and will have the effect of shifting the period. It is common practice to use the prime number divided by 2 rounded up to the next integer. This will have the effect of placing the 0 depth well in the center.

[Acoustics Absorbers & Difussers 3rd Edition](https://www.amazon.com/Acoustic-Absorbers-Diffusers-Trevor-Cox/dp/0367658410), p338

In [4]:
def qr_sequence(w, h, p, k=None):
    if k is None:
        k = np.ceil(p / 2.0)
    return np.array(
        [
            [(((x + k) % w) ** 2 + ((z + k) % w) ** 2) % p for x in range(w)]
            for z in range(h)
        ]
    )

In [5]:
def construct(N, well_width, well_depth):
    # Acoustics Absorbers & Difussers 3rd Edition, p 333
    # w = λ_min / 2
    λ_min = 2 * well_width
    f_max = speed_of_sound / λ_min

    # Acoustics Absorbers & Difussers 3rd Edition, p 336
    sequence = qr_sequence(N, N, N)
    s_max = np.max(sequence)
    f_0 = (s_max / N) * (speed_of_sound / (2 * well_depth))

    # Acoustics Absorbers & Difussers 3rd Edition, p 334
    λ_0 = speed_of_sound / f_0
    d_n = (sequence * λ_0) / (2 * N)

    w_total = N * well_width

    d_min_listening = λ_0 * 3
    d_min_listening

    M = np.max(d_n)
    return {
        # params
        "N": N,
        "well_width": well_width,
        "well_depth": well_depth,
        "well_avg_depth": sum(d_n) / N,
        # results
        "frequency_min": f_0,
        "frequency_max": f_max,
        "wells": d_n,
        "blocks": M - d_n,
        "width": w_total,
    }

In [6]:
# we're assuming squares here
def fit(available_space, well_width, well_depth):
    result = None
    for prime in reversed(primes):
        result = construct(prime, well_width, well_depth)
        if result["width"] < available_space:
            break

    if result["width"] > available_space:
        print("Unable to find a working solution")
        return None

    print(f"N of {result['N']}")
    print(f"Design frequency {result['frequency_min']:.0f}Hz")
    print(f"Upper frequency {result['frequency_max']:.0f}Hz")

    heights = defaultdict(lambda: 0)
    for row in result["blocks"]:
        for block in row:
            heights[block] += 1
    heights[0.0] = 0

    sqr = sum(c * h * well_width for h, c in heights.items())
    total = sum(heights.values())
    print(f"\nConstruction - {total} blocks - {sqr:.3f}m^2")
    print("       mm     in    #  id")
    i = 0
    for height, count in sorted(heights.items()):
        print(f"   {height*1000:6.1f} {height/0.0254:6.2f}  {count:3d} {i:3d}")
        i += 1

    heights = sorted(heights.keys())
    print("Layout")
    for row in result["blocks"]:
        row = [f"{heights.index(b):3d}" for b in row]
        print(" ".join(row))

In [7]:
available_space = (24 * 0.0254) - (2 * 0.019)
print(f"Available width for treatment {available_space}m")

Available width for treatment 0.5715999999999999m


## 1" XPS foam, 6" depth

In [8]:
fit(available_space, 0.0254 * 1, 0.0254 * 6)

N of 19
Design frequency 1066Hz
Upper frequency 6752Hz

Construction - 341 blocks - 0.662m^2
       mm     in    #  id
      0.0   0.00    0   0
      8.5   0.33   20   1
     16.9   0.67   20   2
     25.4   1.00   20   3
     33.9   1.33   20   4
     42.3   1.67   20   5
     50.8   2.00   20   6
     59.3   2.33   20   7
     67.7   2.67   20   8
     76.2   3.00   20   9
     84.7   3.33   20  10
     93.1   3.67   20  11
    101.6   4.00   20  12
    110.1   4.33   20  13
    118.5   4.67   20  14
    127.0   5.00   20  15
    135.5   5.33   20  16
    143.9   5.67   20  17
    152.4   6.00    1  18
Layout
  8   6   2  15   7  16   4   9  12  13  12   9   4  16   7  15   2   6   8
  6   4   0  13   5  14   2   7  10  11  10   7   2  14   5  13   0   4   6
  2   0  15   9   1  10  17   3   6   7   6   3  17  10   1   9  15   0   2
 15  13   9   3  14   4  11  16   0   1   0  16  11   4  14   3   9  13  15
  7   5   1  14   6  15   3   8  11  12  11   8   3  15   6  14   1   5   7


## 2" XPS foam, 6" depth

In [9]:
fit(available_space, 0.0254 * 2, 0.0254 * 6)

N of 11
Design frequency 1023Hz
Upper frequency 3376Hz

Construction - 109 blocks - 0.426m^2
       mm     in    #  id
      0.0   0.00    0   0
     15.2   0.60   12   1
     30.5   1.20   12   2
     45.7   1.80   12   3
     61.0   2.40   12   4
     76.2   3.00   12   5
     91.4   3.60   12   6
    106.7   4.20   12   7
    121.9   4.80   12   8
    137.2   5.40   12   9
    152.4   6.00    1  10
Layout
  4   2   9   3   6   7   6   3   9   2   4
  2   0   7   1   4   5   4   1   7   0   2
  9   7   3   8   0   1   0   8   3   7   9
  3   1   8   2   5   6   5   2   8   1   3
  6   4   0   5   8   9   8   5   0   4   6
  7   5   1   6   9  10   9   6   1   5   7
  6   4   0   5   8   9   8   5   0   4   6
  3   1   8   2   5   6   5   2   8   1   3
  9   7   3   8   0   1   0   8   3   7   9
  2   0   7   1   4   5   4   1   7   0   2
  4   2   9   3   6   7   6   3   9   2   4


## Prototype Diffuser

N=19, above, built out of 1" XPS foam glued to an 1/8th" MDF backer. The whole device weights under 2kg. The process is fairly simple, but does take a decent amount of time, with very little waste. 

<img src="../images/2d-qrd-foam-top.jpg" width=500/>

<img src="../images/2d-qrd-foam-side.jpg" width=500/>

The 4'x8' sheet of XPS was first cut across the 4' dimension into ~36" sections and then split on the breaks with a utility knife. Then the XPS was cut on a [hot wire table](https://www.amazon.com/dp/B0017NS8H6) into 1" strips, resulting in a number of 1"x1"x36" pieces.

A series of spacers were created in [Autodesk Fusion](https://www.autodesk.com/products/fusion-360/overview). The longest matched the size of the 18 piece. A small crosscut sled was built out of scrap and the 18 spacer was placed on it against the wire and a small piece of foam was attached at the other end with double sided tape. One of the strips was placed on the sled against the foam and cut to length. This was the single 18 piece required. The remaining spacers were sized so that when placed against the foam block there would be the desired block length between the wire and the spacer. Each block had its number written on one end with a sharpie marker.

<img src="../images/2d-qrd-foam-cutting.jpg" width=500/>

20x were cut for each of the spacers resulting in the required number of 1-17 blocks.

<img src="../images/2d-qrd-foam-pieces.jpg" width=500/>

An oversized square of 1/8" MDF was cut and two fences covered in packing tape were attached to it at a right angle. The blocks for a row where then arranged on the worktable with the numbers up and double checked taking extra care to ensure 0 height blocks weren't missed. The next step was to spread glue on the numbered end and place it down against the MDF in position. 

<img src="../images/2d-qrd-foam-glueup.jpg" width=500/>